In [1]:
# Author: Mike Hagenow
# Generate some templated behaviors for sanding (including corrections) to be learned through DMP-LWR

import numpy as np
import rospy

from corrective_shared_autonomy.TaskModels.DMPLWRhardcoded import HybridSegment
from corrective_shared_autonomy.TaskModels.DMPLWRhardcoded import DMPLWRhardcoded
from core_robotics.PyBSpline import BSplineSurface
from scipy.spatial.transform import Rotation as ScipyR
from scipy.spatial.transform import Slerp

# starting vals is a list of starting vals
# ending vals is a list of ending vals
# num_pts is how many interp values
# quat_vars is the index of any starts of quaternions so they can be SLERPed instead of LERPed
# super_pos_vars is the starting index for 2-pairs of variables which should have an added sine wave (e.g., rubbing)
# super_pos_freq is the frequency for the starting index pairs
# super_pos_amp is the amplitude for the starting index pairs
def interpMultD(starting_vals,ending_vals,num_pts,quat_vars=[], super_pos_vars = [], super_pos_freq=[], super_pos_amp = []):
    vals = []
    
    # LERP ALL VARIABLES
    for ii in range(0,num_pts):
       
        # super is to potentially add a circular motion on top of the LERP motion
        freq_super = 0.0
        amp_super = 0.0
        added_sine = np.zeros((np.shape(starting_vals)))
        for jj in range(0,len(super_pos_vars)):
            amp = super_pos_amp[jj]
            freq = super_pos_freq[jj]
            added_sine[super_pos_vars[jj]] = np.sin(np.pi*float(ii)/num_pts)*amp*np.sin(freq*float(ii)/2*np.pi)
            added_sine[super_pos_vars[jj]+1] = np.sin(np.pi*float(ii)/num_pts)*amp*np.cos(freq*float(ii)/2*np.pi)
        c_i = float(ii)/float(num_pts) # interpolation coefficient
        vals.append(list(c_i*(e-s)+s+a for s,e,a in zip(starting_vals,ending_vals,added_sine)))
    
    vals = np.asarray(vals) # currently,num_samples x num_vars
    
    # SLERP QUATERIONS
    for jj in range(0,len(quat_vars)):
        starting_q = np.array(starting_vals[quat_vars[jj]:quat_vars[jj]+4])
        ending_q = np.array(ending_vals[quat_vars[jj]:quat_vars[jj]+4])
        key_times = [0,num_pts-1]
        key_rots = ScipyR.from_quat(np.concatenate((starting_q.reshape((1,4)),ending_q.reshape((1,4))),axis=0))
        slerper = Slerp(key_times, key_rots)
        
        pts = list(range(0,num_pts))
        slerped = slerper(pts)
        vals[:,quat_vars[jj]:quat_vars[jj]+4] = np.asarray(slerped.as_quat())
    
    return vals.T

# returns num_vars x num_samples


# starting_vals = np.array([0, 1, 2, 0, 0, 0, 1])
# ending_vals = np.array([1, 2, 3, 0, 0, 0, -1])
# out = interpMultD(starting_vals,ending_vals,10,quat_vars=[3])

# print(out)




In [49]:
##############################
# SCRAP Piece                #
##############################

import rospkg
from scipy.spatial.transform import Rotation as ScipyR
rospack = rospkg.RosPack()
uliconfigpath = rospack.get_path('uli_config')

def gen_approach(surfaceBSpline,samps_per_sec,R_tool_surf,starting_coords,time,tool_offset):
    segment = HybridSegment()
    segment.hybrid = False
    segment.num_samples = int(time*samps_per_sec) # 2 seconds
    segment.state_names = ['x','y','z','qx','qy','qz','qw','delta_s','valve','tool_offset_x','tool_offset_y','tool_offset_z']
    segment.corrections.append(np.zeros((len(segment.state_names),segment.num_samples)))

    approach_pt, n_hat, r_u_norm, r_v_norm = surfaceBSpline.calculate_surface_point(starting_coords[0], starting_coords[1])
    R_surf = ScipyR.from_matrix(np.hstack([r_u_norm.reshape((3,1)), r_v_norm.reshape((3,1)), n_hat.reshape((3,1))]))
    q_app = (R_surf * R_tool_surf).as_quat()

    starting = [approach_pt[0], approach_pt[1], approach_pt[2], q_app[0], q_app[1], q_app[2], q_app[3], 1.0, 0.0, 0.0, 0.0, 0.0]
    ending = [approach_pt[0], approach_pt[1], approach_pt[2], q_app[0], q_app[1], q_app[2], q_app[3], 1.0, 1.0, tool_offset[0], tool_offset[1], tool_offset[2]]
    starting[0:3] = starting[0:3] + 0.1 * n_hat
    ending[0:3] = ending[0:3] + 0.005 * n_hat
    
    orig_vals = interpMultD(starting,ending,segment.num_samples,quat_vars=[3])

    segment.original_vals = []
    segment.original_vals.append(orig_vals)
    
    return segment

def gen_pass(surfaceBSpline,samps_per_sec,R_tool_surf,starting_coords,ending_coords,time, tool_offset):
    segment = HybridSegment()
    segment.hybrid = True
    segment.surface = surfaceBSpline
    segment.num_samples = int(time*samps_per_sec) # 8 seconds
    segment.state_names = ['u','v','f','theta_qx','theta_qy','theta_qz','theta_qw','delta_s','valve','tool_offset_x','tool_offset_y','tool_offset_z']
    segment.original_vals = []

    R_extra_tool = ScipyR.from_euler('x',-15,degrees=True)
    q_extra = R_extra_tool.as_quat()
    corrections = [0.0, 0.0, -5.0, q_extra[0], q_extra[1], q_extra[2], q_extra[3], -0.5, 0.0, 0.02, 0.0, 0.0]
    segment.corrections.append(interpMultD(corrections,corrections,segment.num_samples,quat_vars=[3]))
    # segment.corrections.append(np.zeros((len(segment.state_names),segment.num_samples)))

    app_force = -10.0
    amp = 0.0
    freq = 0.0

    waypoint_0 = interpMultD([starting_coords[0], starting_coords[1], app_force, q_ts[0], q_ts[1], q_ts[2], q_ts[3], 1.0, 1.0, tool_offset[0], tool_offset[1], tool_offset[2]],[ending_coords[0],ending_coords[1], app_force, q_ts[0], q_ts[1], q_ts[2], q_ts[3], 1.0, 1.0, tool_offset[0], tool_offset[1], tool_offset[2]],segment.num_samples,quat_vars=[3], super_pos_vars = [0], super_pos_freq=[freq], super_pos_amp = [amp])
    segment.original_vals.append(waypoint_0)
    
    return segment

def gen_retract(surfaceBSpline,samps_per_sec,R_tool_surf,ending_coords,time, tool_offset):
    segment = HybridSegment()
    segment.hybrid = False
    segment.num_samples = int(time*samps_per_sec) # 2 seconds
    segment.state_names = ['x','y','z','qx','qy','qz','qw','delta_s','valve','tool_offset_x','tool_offset_y','tool_offset_z']
    segment.original_vals = []
    segment.corrections.append(np.zeros((len(segment.state_names),segment.num_samples)))

    retract_pt, n_hat, r_u_norm, r_v_norm = surfaceBSpline.calculate_surface_point(ending_coords[0], ending_coords[1])
    R_surf = ScipyR.from_matrix(np.hstack([r_u_norm.reshape((3,1)), r_v_norm.reshape((3,1)), n_hat.reshape((3,1))]))
    q_ret = (R_surf * R_tool_surf).as_quat()

    starting = [retract_pt[0], retract_pt[1], retract_pt[2], q_ret[0], q_ret[1], q_ret[2], q_ret[3], 1.0, 1.0, tool_offset[0], tool_offset[1], tool_offset[2]]
    ending = [retract_pt[0], retract_pt[1], retract_pt[2], q_ret[0], q_ret[1], q_ret[2], q_ret[3], 1.0, 0.0, 0.0, 0.0, 0.0]
    starting[0:3] = starting[0:3] + 0.005 * n_hat
    ending[0:3] = ending[0:3] + 0.1 * n_hat
    orig_vals = interpMultD(starting,ending,segment.num_samples,quat_vars=[3])
    segment.original_vals.append(orig_vals)
    
    return segment

def gen_btw_passes(surfaceBSpline,samps_per_sec,R_tool_surf,starting_coords,ending_coords,time):
    segment = HybridSegment()
    segment.hybrid = False
    segment.num_samples = int(time*samps_per_sec) # 4 seconds
    segment.state_names = ['x','y','z','qx','qy','qz','qw','delta_s','valve']
    segment.original_vals = []
    segment.corrections.append(np.zeros((len(segment.state_names),segment.num_samples)))

    retract_pt, n_hat1, r_u_norm, r_v_norm = surfaceBSpline.calculate_surface_point(starting_coords[0], starting_coords[1])
    R_surf = ScipyR.from_matrix(np.hstack([r_u_norm.reshape((3,1)), r_v_norm.reshape((3,1)), n_hat.reshape((3,1))]))
    q_ret = (R_surf * R_tool_surf).as_quat()
    
    approach_pt, n_hat2, r_u_norm, r_v_norm = surfaceBSpline.calculate_surface_point(ending_coords[0], ending_coords[1])
    R_surf = ScipyR.from_matrix(np.hstack([r_u_norm.reshape((3,1)), r_v_norm.reshape((3,1)), n_hat.reshape((3,1))]))
    q_app = (R_surf * R_tool_surf).as_quat()
    
    starting = [retract_pt[0], retract_pt[1], retract_pt[2], q_ret[0], q_ret[1], q_ret[2], q_ret[3], 1.0, 0.0]
    starting[0:3] = starting[0:3] + 0.1 * n_hat1
    ending = [approach_pt[0], approach_pt[1], approach_pt[2], q_app[0], q_app[1], q_app[2], q_app[3], 1.0, 0.0]
    ending[0:3] = ending[0:3] + 0.1 * n_hat2
    orig_vals = interpMultD(starting,ending,segment.num_samples,quat_vars=[3])
    segment.original_vals.append(orig_vals)
    
    return segment


####################
# Set up behavior  #
####################
samps_per_sec = 40

surfaceBSplineFile = uliconfigpath+'/registration_models/IRC_piece1.csv'
surfaceBSpline = BSplineSurface()
surfaceBSpline.loadSurface(surfaceBSplineFile)

segments = []
ang_relative_to_surface = 90 # degrees
R_tool_surf = ScipyR.from_euler('ZY',[ang_relative_to_surface,0],degrees=True)
q_ts = R_tool_surf.as_quat()
    
starting_passes = [[0.05,0.1],[0.35,0.1],[0.7,0.1]]
ending_passes = [[0.05,0.8],[0.35,0.8],[0.7,0.8]]
# starting_passes = [[0.1, 0.1]]
# ending_passes = [[0.1,0.8]]
pass_tool_offsets = [[0.0, 0.0, 0.0],[0.0, 0.0, 0.0],[0.0, 0.0, 0.0]]

for ii in range(0,len(starting_passes)):
    segmentTemp = gen_approach(surfaceBSpline,samps_per_sec,R_tool_surf,starting_passes[ii],1.0,pass_tool_offsets[ii])
    segments.append(segmentTemp)
    segmentTemp = gen_pass(surfaceBSpline,samps_per_sec,R_tool_surf,starting_passes[ii],ending_passes[ii],20.0,pass_tool_offsets[ii])
    segments.append(segmentTemp)
    segmentTemp = gen_retract(surfaceBSpline,samps_per_sec,R_tool_surf,ending_passes[ii],1.0,pass_tool_offsets[ii])
    segments.append(segmentTemp)
    if ii != len(starting_passes)-1:
        segmentTemp = gen_btw_passes(surfaceBSpline,samps_per_sec,R_tool_surf,ending_passes[ii],starting_passes[ii+1],3.0)
        segments.append(segmentTemp)



In [2]:
# ########################################
# # The Sandening                       ##
# ########################################

# starting_surf = [0.1, 0.2]
# ending_surf = [0.1, 0.8]
# num_passes = 1

# samps_per_sec = 40

# import rospkg
# rospack = rospkg.RosPack()
# uliconfigpath = rospack.get_path('uli_config')


# surfaceBSplineFile = uliconfigpath+'/registration_models/IRC_piece1.csv'
# surfaceBSpline = BSplineSurface()
# surfaceBSpline.loadSurface(surfaceBSplineFile)

# segments = []

# from scipy.spatial.transform import Rotation as ScipyR
# ang_relative_to_surface = 90 # degrees
# R_tool_surf = ScipyR.from_euler('ZY',[ang_relative_to_surface,0],degrees=True)

# q_ts = R_tool_surf.as_quat()
# print(q_ts)

# # Part 1: approach the starting point
# segment1 = HybridSegment()
# segment1.hybrid = False
# segment1.num_samples = 6*samps_per_sec # 2 seconds
# segment1.state_names = ['x','y','z','qx','qy','qz','qw','delta_s','valve']
# segment1.corrections.append(np.zeros((len(segment1.state_names),segment1.num_samples)))

# approach_pt, n_hat, r_u_norm, r_v_norm = surfaceBSpline.calculate_surface_point(starting_surf[0], starting_surf[1])
# R_surf = ScipyR.from_matrix(np.hstack([r_u_norm.reshape((3,1)), r_v_norm.reshape((3,1)), n_hat.reshape((3,1))]))
# q_app = (R_surf * R_tool_surf).as_quat()
# if q_app[3]<0:
#     q_app*=-1.0

# print("QAPP: ",q_app)

# starting = [approach_pt[0], approach_pt[1], approach_pt[2] + 0.1, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0]
# ending = [approach_pt[0], approach_pt[1], approach_pt[2] + 0.005, q_app[0], q_app[1], q_app[2], q_app[3], 1.0, 1.0]
# orig_vals = interpMultD(starting,ending,segment1.num_samples,quat_vars=[3])


# # number of state variables by number of samples

# segment1.original_vals = []
# segment1.original_vals.append(orig_vals)

# segments.append(segment1)


# # Part 2: one pass over surface
# segment2 = HybridSegment()
# segment2.hybrid = True
# segment2.surface = surfaceBSpline
# segment2.num_samples = 10*samps_per_sec # 8 seconds
# segment2.state_names = ['u','v','f','theta_qx','theta_qy','theta_qz','theta_qw','delta_s','valve']
# segment2.original_vals = []

# R_extra_tool = ScipyR.from_euler('x',-15,degrees=True)
# q_extra = R_extra_tool.as_quat()
# corrections = [0.0, 0.0, -5.0, q_extra[0], q_extra[1], q_extra[2], q_extra[3], -0.5, 0.0]
# segment2.corrections.append(interpMultD(corrections,corrections,segment2.num_samples,quat_vars=[3]))
# # segment2.corrections.append(np.zeros((len(segment2.state_names),segment2.num_samples)))

# samps = int(segment2.num_samples / 5)
# app_force = -10.0
# amp = 0.0
# freq = 0.0

# waypoint_0 = interpMultD([starting_surf[0], starting_surf[1], app_force, q_ts[0], q_ts[1], q_ts[2], q_ts[3], 1.0, 1.0],[ending_surf[0],ending_surf[1], app_force, q_ts[0], q_ts[1], q_ts[2], q_ts[3], 1.0, 1.0],segment2.num_samples,quat_vars=[3], super_pos_vars = [0], super_pos_freq=[freq], super_pos_amp = [amp])
# segment2.original_vals.append(waypoint_0)

# segments.append(segment2)


# # # Part 3: move away from the surface
# segment3 = HybridSegment()
# segment3.hybrid = False
# segment3.num_samples = 2*samps_per_sec # 2 seconds
# segment3.state_names = ['x','y','z','qx','qy','qz','qw','delta_s','valve']
# segment3.original_vals = []
# segment3.corrections.append(np.zeros((len(segment3.state_names),segment3.num_samples)))

# retract_pt, n_hat, r_u_norm, r_v_norm = surfaceBSpline.calculate_surface_point(ending_surf[0], ending_surf[1])
# R_surf = ScipyR.from_matrix(np.hstack([r_u_norm.reshape((3,1)), r_v_norm.reshape((3,1)), n_hat.reshape((3,1))]))
# q_ret = (R_surf * R_tool_surf).as_quat()

# starting = [retract_pt[0], retract_pt[1], retract_pt[2]+ 0.005, q_ret[0], q_ret[1], q_ret[2], q_ret[3], 1.0, 1.0]
# ending = [retract_pt[0], retract_pt[1], retract_pt[2] + 0.1, q_ret[0], q_ret[1], q_ret[2], q_ret[3], 1.0, 0.0]
# orig_vals = interpMultD(starting,ending,segment3.num_samples,quat_vars=[3])
# segment3.original_vals.append(orig_vals)

# segments.append(segment3)


# # # Part 4: move from end to start of next pass
# segment4 = HybridSegment()
# segment4.hybrid = False
# segment4.num_samples = 4*samps_per_sec # 4 seconds
# segment4.state_names = ['x','y','z','qx','qy','qz','qw','delta_s','valve']
# segment4.original_vals = []
# segment4.corrections.append(np.zeros((len(segment4.state_names),segment4.num_samples)))

# retract_pt, n_hat, r_u_norm, r_v_norm = surfaceBSpline.calculate_surface_point(ending_surf[0], ending_surf[1])
# R_surf = ScipyR.from_matrix(np.hstack([r_u_norm.reshape((3,1)), r_v_norm.reshape((3,1)), n_hat.reshape((3,1))]))
# q_ret = (R_surf * R_tool_surf).as_quat()
# if q_ret[3]<0:
#     q_ret*=-1.0

# starting = [retract_pt[0], retract_pt[1], retract_pt[2] + 0.1, q_ret[0], q_ret[1], q_ret[2], q_ret[3], 1.0, 0.0]
# ending = [approach_pt[0], approach_pt[1], approach_pt[2] + 0.1, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0]
# orig_vals = interpMultD(starting,ending,segment4.num_samples,quat_vars=[3])
# segment4.original_vals.append(orig_vals)

# segments.append(segment4)

# for ii in range(0,num_passes-2):
#     segments.extend(segments[0:4])
# segments.extend(segments[:3])

[0.         0.         0.70710678 0.70710678]
QAPP:  [ 8.15450596e-04 -1.99158424e-01  9.92900872e-03  9.79916666e-01]


In [50]:
############################
# Learn IRC1 model         #
############################
import rospkg
rospack = rospkg.RosPack()
config_dir = rospack.get_path('uli_config')+'/registration_models/'


model = DMPLWRhardcoded(verbose=True, dt=1/samps_per_sec)
learnedSegments = model.learnModel(segments,config_dir+'IRC_piece1.pkl') # second argument is the outfile


-------------------
Model Learning: DMP + LWR Hardcoded
-------------------
DMP for segment  0  of  10
DMP for segment  1  of  10
DMP for segment  2  of  10
DMP for segment  3  of  10
DMP for segment  4  of  10
DMP for segment  5  of  10
DMP for segment  6  of  10
DMP for segment  7  of  10
DMP for segment  8  of  10
DMP for segment  9  of  10
DMP for segment  10  of  10
Time to learn model: 23.38949966430664 seconds


[WARN] [1653515593.210436]: Inbound TCP/IP connection failed: connection from sender terminated before handshake header received. 0 bytes were received. Please check sender for additional details.
[WARN] [1653515593.215900]: Inbound TCP/IP connection failed: connection from sender terminated before handshake header received. 0 bytes were received. Please check sender for additional details.


In [35]:
# ############################
# # Execute IRC1             #
# ############################

# rospy.init_node('executeROSfromState', anonymous=True)

# r_surf = np.array([0.4091875321237706, 0.3659806127623546, 0.6255911510276543, -0.5543099010413256])
# t_surf = np.array([0.34226860621143673, -0.22084696598873957, 0.281413347966159])

# model.executeModel(learnedSegments=learnedSegments, R_surface = r_surf, t_surface = t_surf, input_type='1dof')

not hybrid


[WARN] [1653513731.315923]: Inbound TCP/IP connection failed: connection from sender terminated before handshake header received. 0 bytes were received. Please check sender for additional details.
[WARN] [1653513731.809152]: Inbound TCP/IP connection failed: connection from sender terminated before handshake header received. 0 bytes were received. Please check sender for additional details.
[WARN] [1653513731.813643]: Inbound TCP/IP connection failed: connection from sender terminated before handshake header received. 0 bytes were received. Please check sender for additional details.


In [ ]:
# ########################################
# # IRC BEHAVIOR 1 - with superposition ##
# ########################################

# # Behavior three: Three passes on the IRC surface (slow with corrections)

# import rospkg
# rospack = rospkg.RosPack()
# uliconfigpath = rospack.get_path('uli_config')


# surfaceBSplineFile = uliconfigpath+'/registration_models/IRC_piece1.csv'
# surfaceBSpline = BSplineSurface()
# surfaceBSpline.loadSurface(surfaceBSplineFile)

# segments = []

# from scipy.spatial.transform import Rotation as ScipyR
# ang_relative_to_surface = 90 # degrees
# R_tool_surf = ScipyR.from_euler('z',ang_relative_to_surface,degrees=True)
# q_ts = R_tool_surf.as_quat()


# # Part 1: approach the starting point
# segment1 = HybridSegment()
# segment1.hybrid = False
# segment1.num_samples = 600 # 2 seconds
# segment1.state_names = ['x','y','z','qx','qy','qz','qw','delta_s','valve']
# segment1.corrections.append(np.zeros((len(segment1.state_names),segment1.num_samples)))

# approach_pt, n_hat, r_u_norm, r_v_norm = surfaceBSpline.calculate_surface_point(0.1, 0.2)
# R_surf = ScipyR.from_matrix(np.hstack([r_u_norm.reshape((3,1)), r_v_norm.reshape((3,1)), n_hat.reshape((3,1))]))
# q_app = (R_surf * R_tool_surf).as_quat()
# if q_app[3]<0:
#     q_app*=-1.0

# print("QAPP: ",q_app)

# starting = [approach_pt[0], approach_pt[1], approach_pt[2] + 0.1, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0]
# ending = [approach_pt[0], approach_pt[1], approach_pt[2] + 0.01, q_app[0], q_app[1], q_app[2], q_app[3], 1.0, 1.0]
# orig_vals = interpMultD(starting,ending,segment1.num_samples,quat_vars=[3])


# # number of state variables by number of samples

# segment1.original_vals = []
# segment1.original_vals.append(orig_vals)

# segments.append(segment1)


# # Part 2: one pass over surface
# segment2 = HybridSegment()
# segment2.hybrid = True
# segment2.surface = surfaceBSpline
# segment2.num_samples = 2000 # 8 seconds
# segment2.state_names = ['u','v','f','theta_qx','theta_qy','theta_qz','theta_qw','delta_s','valve']
# segment2.original_vals = []

# R_extra_tool = ScipyR.from_euler('y',5,degrees=True)
# q_extra = R_extra_tool.as_quat()
# corrections = [0.0, 0.0, -5.0, q_extra[0], q_extra[1], q_extra[2], q_extra[3], -1.5, 0.0]
# segment2.corrections.append(interpMultD(corrections,corrections,segment2.num_samples,quat_vars=[3]))


# samps = int(segment2.num_samples / 5)
# app_force = -8.0
# amp = 0.05
# freq = 0.2
# waypoint_0 = interpMultD([0.1, 0.2, app_force, q_ts[0], q_ts[1], q_ts[2], q_ts[3], 1.0, 1.0],[0.1, 0.5, app_force, q_ts[0], q_ts[1], q_ts[2], q_ts[3], 1.0, 1.0],samps,quat_vars=[3], super_pos_vars = [0], super_pos_freq=[freq], super_pos_amp = [amp])
# waypoint_1 = interpMultD([0.1, 0.5, app_force, q_ts[0], q_ts[1], q_ts[2], q_ts[3], 1.0, 1.0],[0.5, 0.5, app_force, q_ts[0], q_ts[1], q_ts[2], q_ts[3], 1.0, 1.0],samps,quat_vars=[3], super_pos_vars = [0], super_pos_freq=[freq], super_pos_amp = [amp])
# waypoint_2 = interpMultD([0.5, 0.5, app_force, q_ts[0], q_ts[1], q_ts[2], q_ts[3], 1.0, 1.0],[0.5, 0.2, app_force, q_ts[0], q_ts[1], q_ts[2], q_ts[3], 1.0, 1.0],samps,quat_vars=[3], super_pos_vars = [0], super_pos_freq=[freq], super_pos_amp = [amp])
# waypoint_3 = interpMultD([0.5, 0.2, app_force, q_ts[0], q_ts[1], q_ts[2], q_ts[3], 1.0, 1.0],[0.9, 0.2, app_force, q_ts[0], q_ts[1], q_ts[2], q_ts[3], 1.0, 1.0],samps,quat_vars=[3], super_pos_vars = [0], super_pos_freq=[freq], super_pos_amp = [amp])
# waypoint_4 = interpMultD([0.9, 0.2, app_force, q_ts[0], q_ts[1], q_ts[2], q_ts[3], 1.0, 1.0],[0.9, 0.5, app_force, q_ts[0], q_ts[1], q_ts[2], q_ts[3], 1.0, 1.0],samps,quat_vars=[3], super_pos_vars = [0], super_pos_freq=[freq], super_pos_amp = [amp])

# waypoints = np.hstack([waypoint_0,waypoint_1,waypoint_2,waypoint_3,waypoint_4])

# segment2.original_vals.append(waypoints)

# segments.append(segment2)


# # # Part 3: move away from the surface
# segment3 = HybridSegment()
# segment3.hybrid = False
# segment3.num_samples = 200 # 2 seconds
# segment3.state_names = ['x','y','z','qx','qy','qz','qw','delta_s','valve']
# segment3.original_vals = []
# segment3.corrections.append(np.zeros((len(segment3.state_names),segment3.num_samples)))

# retract_pt, n_hat, r_u_norm, r_v_norm = surfaceBSpline.calculate_surface_point(0.9, 0.5)
# R_surf = ScipyR.from_matrix(np.hstack([r_u_norm.reshape((3,1)), r_v_norm.reshape((3,1)), n_hat.reshape((3,1))]))
# q_ret = (R_surf * R_tool_surf).as_quat()

# starting = [retract_pt[0], retract_pt[1], retract_pt[2]+ 0.01, q_ret[0], q_ret[1], q_ret[2], q_ret[3], 1.0, 1.0]
# ending = [retract_pt[0], retract_pt[1], retract_pt[2] + 0.1, q_ret[0], q_ret[1], q_ret[2], q_ret[3], 1.0, 0.0]
# orig_vals = interpMultD(starting,ending,segment3.num_samples,quat_vars=[3])
# segment3.original_vals.append(orig_vals)

# segments.append(segment3)


# # # Part 4: move from end to start of next pass
# segment4 = HybridSegment()
# segment4.hybrid = False
# segment4.num_samples = 400 # 4 seconds
# segment4.state_names = ['x','y','z','qx','qy','qz','qw','delta_s','valve']
# segment4.original_vals = []
# segment4.corrections.append(np.zeros((len(segment4.state_names),segment4.num_samples)))

# retract_pt, n_hat, r_u_norm, r_v_norm = surfaceBSpline.calculate_surface_point(0.9, 0.5)
# R_surf = ScipyR.from_matrix(np.hstack([r_u_norm.reshape((3,1)), r_v_norm.reshape((3,1)), n_hat.reshape((3,1))]))
# q_ret = (R_surf * R_tool_surf).as_quat()
# if q_ret[3]<0:
#     q_ret*=-1.0

# starting = [retract_pt[0], retract_pt[1], retract_pt[2] + 0.1, q_ret[0], q_ret[1], q_ret[2], q_ret[3], 1.0, 0.0]
# ending = [approach_pt[0], approach_pt[1], approach_pt[2] + 0.1, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0]
# orig_vals = interpMultD(starting,ending,segment4.num_samples,quat_vars=[3])
# segment4.original_vals.append(orig_vals)

# segments.append(segment4)

# segments.extend(segments)
# segments.extend(segments[:3])

In [51]:
##############################
# Anna Behavior              #
##############################

import rospkg
from scipy.spatial.transform import Rotation as ScipyR
rospack = rospkg.RosPack()
uliconfigpath = rospack.get_path('uli_config')

def gen_approach(surfaceBSpline,samps_per_sec,R_tool_surf,starting_coords,time,tool_offset):
    segment = HybridSegment()
    segment.hybrid = False
    segment.num_samples = int(time*samps_per_sec) # 2 seconds
    segment.state_names = ['x','y','z','qx','qy','qz','qw','delta_s','valve','tool_offset_x','tool_offset_y','tool_offset_z']
    segment.corrections.append(np.zeros((len(segment.state_names),segment.num_samples)))

    approach_pt, n_hat, r_u_norm, r_v_norm = surfaceBSpline.calculate_surface_point(starting_coords[0], starting_coords[1])
    R_surf = ScipyR.from_matrix(np.hstack([r_u_norm.reshape((3,1)), r_v_norm.reshape((3,1)), n_hat.reshape((3,1))]))
    q_app = (R_surf * R_tool_surf).as_quat()

    starting = [approach_pt[0], approach_pt[1], approach_pt[2], q_app[0], q_app[1], q_app[2], q_app[3], 1.0, 0.0, 0.0, 0.0, 0.0]
    ending = [approach_pt[0], approach_pt[1], approach_pt[2], q_app[0], q_app[1], q_app[2], q_app[3], 1.0, 1.0, tool_offset[0], tool_offset[1], tool_offset[2]]
    starting[0:3] = starting[0:3] + 0.1 * n_hat
    ending[0:3] = ending[0:3] + 0.005 * n_hat
    
    orig_vals = interpMultD(starting,ending,segment.num_samples,quat_vars=[3])

    segment.original_vals = []
    segment.original_vals.append(orig_vals)
    
    return segment

def gen_pass(surfaceBSpline,samps_per_sec,R_tool_surf,starting_coords,ending_coords,time, tool_offset):
    segment = HybridSegment()
    segment.hybrid = True
    segment.surface = surfaceBSpline
    segment.num_samples = int(time*samps_per_sec) # 8 seconds
    segment.state_names = ['u','v','f','theta_qx','theta_qy','theta_qz','theta_qw','delta_s','valve','tool_offset_x','tool_offset_y','tool_offset_z']
    segment.original_vals = []

    R_extra_tool = ScipyR.from_euler('y',-10,degrees=True)
    q_extra = R_extra_tool.as_quat()
    corrections = [0.0, 0.0, -5.0, q_extra[0], q_extra[1], q_extra[2], q_extra[3], -0.5, 0.0, 0.0, 0.0, 0.0]
    segment.corrections.append(interpMultD(corrections,corrections,segment.num_samples,quat_vars=[3]))
    # segment.corrections.append(np.zeros((len(segment.state_names),segment.num_samples)))

    app_force = -10.0
    amp = 0.0
    freq = 0.0

    waypoint_0 = interpMultD([starting_coords[0], starting_coords[1], app_force, q_ts[0], q_ts[1], q_ts[2], q_ts[3], 1.0, 1.0, tool_offset[0], tool_offset[1], tool_offset[2]],[ending_coords[0],ending_coords[1], app_force, q_ts[0], q_ts[1], q_ts[2], q_ts[3], 1.0, 1.0, tool_offset[0], tool_offset[1], tool_offset[2]],segment.num_samples,quat_vars=[3], super_pos_vars = [0], super_pos_freq=[freq], super_pos_amp = [amp])
    segment.original_vals.append(waypoint_0)
    
    return segment

def gen_retract(surfaceBSpline,samps_per_sec,R_tool_surf,ending_coords,time, tool_offset):
    segment = HybridSegment()
    segment.hybrid = False
    segment.num_samples = int(time*samps_per_sec) # 2 seconds
    segment.state_names = ['x','y','z','qx','qy','qz','qw','delta_s','valve','tool_offset_x','tool_offset_y','tool_offset_z']
    segment.original_vals = []
    segment.corrections.append(np.zeros((len(segment.state_names),segment.num_samples)))

    retract_pt, n_hat, r_u_norm, r_v_norm = surfaceBSpline.calculate_surface_point(ending_coords[0], ending_coords[1])
    R_surf = ScipyR.from_matrix(np.hstack([r_u_norm.reshape((3,1)), r_v_norm.reshape((3,1)), n_hat.reshape((3,1))]))
    q_ret = (R_surf * R_tool_surf).as_quat()

    starting = [retract_pt[0], retract_pt[1], retract_pt[2], q_ret[0], q_ret[1], q_ret[2], q_ret[3], 1.0, 1.0, tool_offset[0], tool_offset[1], tool_offset[2]]
    ending = [retract_pt[0], retract_pt[1], retract_pt[2], q_ret[0], q_ret[1], q_ret[2], q_ret[3], 1.0, 0.0, 0.0, 0.0, 0.0]
    starting[0:3] = starting[0:3] + 0.005 * n_hat
    ending[0:3] = ending[0:3] + 0.1 * n_hat
    orig_vals = interpMultD(starting,ending,segment.num_samples,quat_vars=[3])
    segment.original_vals.append(orig_vals)
    
    return segment

def gen_btw_passes(surfaceBSpline,samps_per_sec,R_tool_surf,starting_coords,ending_coords,time):
    segment = HybridSegment()
    segment.hybrid = False
    segment.num_samples = int(time*samps_per_sec) # 4 seconds
    segment.state_names = ['x','y','z','qx','qy','qz','qw','delta_s','valve']
    segment.original_vals = []
    segment.corrections.append(np.zeros((len(segment.state_names),segment.num_samples)))

    retract_pt, n_hat, r_u_norm, r_v_norm = surfaceBSpline.calculate_surface_point(starting_coords[0], starting_coords[1])
    R_surf = ScipyR.from_matrix(np.hstack([r_u_norm.reshape((3,1)), r_v_norm.reshape((3,1)), n_hat.reshape((3,1))]))
    q_ret = (R_surf * R_tool_surf).as_quat()
    
    approach_pt, n_hat, r_u_norm, r_v_norm = surfaceBSpline.calculate_surface_point(ending_coords[0], ending_coords[1])
    R_surf = ScipyR.from_matrix(np.hstack([r_u_norm.reshape((3,1)), r_v_norm.reshape((3,1)), n_hat.reshape((3,1))]))
    q_app = (R_surf * R_tool_surf).as_quat()
    
    starting = [retract_pt[0], retract_pt[1], retract_pt[2], q_ret[0], q_ret[1], q_ret[2], q_ret[3], 1.0, 0.0]
    starting[0:3] = starting[0:3] + 0.1 * n_hat
    ending = [approach_pt[0], approach_pt[1], approach_pt[2], q_app[0], q_app[1], q_app[2], q_app[3], 1.0, 0.0]
    ending[0:3] = ending[0:3] + 0.1 * n_hat
    orig_vals = interpMultD(starting,ending,segment.num_samples,quat_vars=[3])
    segment.original_vals.append(orig_vals)
    
    return segment


####################
# Set up behavior  #
####################
samps_per_sec = 40

surfaceBSplineFile = uliconfigpath+'/registration_models/anna_study1.csv'
surfaceBSpline = BSplineSurface()
surfaceBSpline.loadSurface(surfaceBSplineFile)

segments = []
ang_relative_to_surface = 0 # degrees
R_tool_surf = ScipyR.from_euler('ZY',[ang_relative_to_surface,0],degrees=True)
q_ts = R_tool_surf.as_quat()
    
# starting_passes = [[0.9,0.3],[0.9,0.5],[0.9,0.7]]
# ending_passes = [[0.1,0.3],[0.1,0.5],[0.1,0.7]]
starting_passes = [[0.1, 0.5]]
ending_passes = [[0.9,0.5]]
pass_tool_offsets = [[0.0, -0.03, 0.0]]

for ii in range(0,len(starting_passes)):
    segmentTemp = gen_approach(surfaceBSpline,samps_per_sec,R_tool_surf,starting_passes[ii],3.0,pass_tool_offsets[ii])
    segments.append(segmentTemp)
    segmentTemp = gen_pass(surfaceBSpline,samps_per_sec,R_tool_surf,starting_passes[ii],ending_passes[ii],3.0,pass_tool_offsets[ii])
    segments.append(segmentTemp)
    segmentTemp = gen_retract(surfaceBSpline,samps_per_sec,R_tool_surf,ending_passes[ii],1.0,pass_tool_offsets[ii])
    segments.append(segmentTemp)
    if ii != len(starting_passes)-1:
        segmentTemp = gen_btw_passes(surfaceBSpline,samps_per_sec,R_tool_surf,ending_passes[ii],starting_passes[ii+1],1.0)
        segments.append(segmentTemp)



# for ii in range(0,num_passes-2):
#     segments.extend(segments[0:4])
# segments.extend(segments[:3])


In [52]:
############################
# Learn anna model         #
############################
import rospkg
rospack = rospkg.RosPack()
config_dir = rospack.get_path('uli_config')+'/registration_models/'


model = DMPLWRhardcoded(verbose=True, dt=1/samps_per_sec)
learnedSegments = model.learnModel(segments,config_dir+'anna_study1.pkl') # second argument is the outfile


-------------------
Model Learning: DMP + LWR Hardcoded
-------------------
DMP for segment  0  of  2
DMP for segment  1  of  2
DMP for segment  2  of  2
Time to learn model: 6.618419408798218 seconds


In [54]:
############################
# Execute anna model       #
############################

rospy.init_node('executeROSfromState', anonymous=True)

r_surf = np.array([-0.004334668895692079, -0.0073819671090338814, -0.02734169882099563, 0.9995894900971837])
t_surf = np.array([0.4836419041526589, 0.13975673119326526, 0.1548760284325566])

model.executeModel(learnedSegments=learnedSegments, R_surface = r_surf, t_surface = t_surf, input_type='1dof')

not hybrid
hybrid
not hybrid


[WARN] [1653516466.040379]: Inbound TCP/IP connection failed: connection from sender terminated before handshake header received. 0 bytes were received. Please check sender for additional details.
[WARN] [1653516481.824157]: Inbound TCP/IP connection failed: connection from sender terminated before handshake header received. 0 bytes were received. Please check sender for additional details.
[WARN] [1653516519.764884]: Inbound TCP/IP connection failed: connection from sender terminated before handshake header received. 0 bytes were received. Please check sender for additional details.
[WARN] [1653516528.859543]: Inbound TCP/IP connection failed: connection from sender terminated before handshake header received. 0 bytes were received. Please check sender for additional details.
[WARN] [1653516528.868649]: Inbound TCP/IP connection failed: connection from sender terminated before handshake header received. 0 bytes were received. Please check sender for additional details.
[WARN] [1653516